<a href="https://colab.research.google.com/github/takedatmh/toyama/blob/main/2_9_PreTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/ColabNotebooks/Toyama_University/PreTrain"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset, Dataset

# 1. モデルとトークナイザのロード
model_name = "rinna/japanese-gpt2-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2. 継続学習させる日本語データ（例：テスト用の短文）
texts = [
    "日本の首都は東京です。",
    "富士山は日本で最も高い山です。",
    "りんなは日本語に特化した大規模言語モデルです。"
]

dataset = Dataset.from_dict({"text": texts})

# 3. トークン化
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 4. データコラレータ（Language Modeling用）
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # GPT系はMaskedではなくCausal
)

# 5. 学習設定
training_args = TrainingArguments(
    output_dir="./rinna-continued",
    per_device_train_batch_size=2,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=1,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

# 6. トレーナー構築と学習開始
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/806k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

model.safetensors:   0%|          | 0.00/454M [00:00<?, ?B/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,5.813800
2,4.122400
3,4.338000
4,6.175800
5,4.405600
6,3.011600
7,3.850900
8,2.804800
9,3.422900
10,2.446100


TrainOutput(global_step=10, training_loss=4.039201068878174, metrics={'train_runtime': 27.5584, 'train_samples_per_second': 0.544, 'train_steps_per_second': 0.363, 'total_flos': 979845120000.0, 'train_loss': 4.039201068878174, 'epoch': 5.0})

In [ ]:
# モデルとトークナイザを保存（Google Drive）
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('/content/drive/MyDrive/ColabNotebooks/Toyama_University/PreTrain/tokenizer_config.json',
 '/content/drive/MyDrive/ColabNotebooks/Toyama_University/PreTrain/special_tokens_map.json',
 '/content/drive/MyDrive/ColabNotebooks/Toyama_University/PreTrain/spiece.model',
 '/content/drive/MyDrive/ColabNotebooks/Toyama_University/PreTrain/added_tokens.json',
 '/content/drive/MyDrive/ColabNotebooks/Toyama_University/PreTrain/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# 保存済みモデルとトークナイザをロード
model = AutoModelForCausalLM.from_pretrained(save_path).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(save_path)

# 推論用プロンプト
prompt = "りんなは、どのような言語モデルですか？"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# 推論設定
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )

# 結果表示
generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("🧠 モデルの回答:", generated.replace(prompt, "").strip())


🧠 モデルの回答: りんなは、どのような言語モデルですか?なぜ私たちの学校では、プログラミングやデザインといった「学び」をどのように学んでいますか? it(情報技術)教育はどうやって生まれていますか? なぜプログラミングやデザインは生まれたのでしょうか。 「学び」とはどんなものでしょうか? 授業の中で教えて下さった生徒さんに伝えたいことはありますか? 私自身が学んだ知識だけでは十分ですが、お客様から得た情報をより効果的に活用するために実践しているスキルなどを教えていただけますか
